In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, torch

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# device0 = "cuda:0" if torch.cuda.is_available() else "cpu"
device7 = "cuda:0" if torch.cuda.is_available() else "cpu"
OCCUPY_CUDA_7 = torch.randn(1, 1, device=device7)


In [5]:

import csv
sample_path = 'reference_data/sample_pfda/ValidationDataset-sampledata.csv'
rectified_path = 'data/rectified_dataset_fog_release.csv'
with open(sample_path, mode ='r')as file:
    csvFile = csv.reader(file)
    count = 0
    for lines in csvFile:
        if count == 2:
                break
        print(lines)
        count+=1
        

['Time', 'GeneralEvent', 'ClinicalEvent', 'L Foot Contact', 'R Foot Contact', 'L Foot Pressure', 'R Foot Pressure', 'Walkway_X', 'Walkway_Y', 'WalkwayPressureLevel', 'WalkwayFoot', 'LowerBack_Acc_X', 'LowerBack_Acc_Y', 'LowerBack_Acc_Z', 'LowerBack_FreeAcc_E', 'LowerBack_FreeAcc_N', 'LowerBack_FreeAcc_U', 'LowerBack_Gyr_X', 'LowerBack_Gyr_Y', 'LowerBack_Gyr_Z', 'LowerBack_Mag_X', 'LowerBack_Mag_Y', 'LowerBack_Mag_Z', 'LowerBack_VelInc_X', 'LowerBack_VelInc_Y', 'LowerBack_VelInc_Z', 'LowerBack_OriInc_q0', 'LowerBack_OriInc_q1', 'LowerBack_OriInc_q2', 'LowerBack_OriInc_q3', 'LowerBack_Roll', 'LowerBack_Pitch', 'LowerBack_Yaw', 'R_Wrist_Acc_X', 'R_Wrist_Acc_Y', 'R_Wrist_Acc_Z', 'R_Wrist_FreeAcc_E', 'R_Wrist_FreeAcc_N', 'R_Wrist_FreeAcc_U', 'R_Wrist_Gyr_X', 'R_Wrist_Gyr_Y', 'R_Wrist_Gyr_Z', 'R_Wrist_Mag_X', 'R_Wrist_Mag_Y', 'R_Wrist_Mag_Z', 'R_Wrist_VelInc_X', 'R_Wrist_VelInc_Y', 'R_Wrist_VelInc_Z', 'R_Wrist_OriInc_q0', 'R_Wrist_OriInc_q1', 'R_Wrist_OriInc_q2', 'R_Wrist_OriInc_q3', 'R_Wris

In [10]:
data_folder = os.listdir('data')
rectified_csv_files = [file for file in data_folder if file.startswith('rec') \
                                                        and file.endswith('.csv')]
print(rectified_csv_files)
rectified_csv_files = sorted(rectified_csv_files, key=lambda x: (int(x[:-4].split('_')[1])))
rectified_csv_files

['rec_1.csv', 'rec_2.csv', 'rec_11.csv']


['rec_1.csv', 'rec_2.csv', 'rec_11.csv']